In [ ]:
!pip install onednn-cpu-gomp
import oneDNN as dnn
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
os.environ['DNNL_ENGINE_LIMIT_CPU_CAPABILITIES'] = '0'
os.environ['DNNL_VERBOSE'] = '1'
os.environ['SYCL_DEVICE_FILTER'] = 'opencl:gpu'

In [ ]:
!pip install scikit-learn-intelex
from sklearn import patch_sklearn
from sklearn.preprocessing import StandardScaler
patch_sklearn()

In [ ]:
import os
import numpy as np
import cv2
import imghdr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from daal4py import normalization_zscore

funct to load using OneDAL


In [ ]:
def load_and_preprocess_image(image_path):
    try:
        img = cv2.imread(image_path)
        tip = imghdr.what(image_path)
        if tip not in image_exts:
            print("Image not in ext list{}".format(image_path))
            os.remove(image_path)
            return None
        # Resize the image to desired dimensions (e.g., 256x256)
        img_resized = cv2.resize(img, (256, 256))
        return img_resized
    except Exception as e:
        print('issue with image{}'.format(image_path))
        return None

In [ ]:
data_dir = 'data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

# Load and preprocess images using OneDAL
X_data = []
y_data = []
for image_class in os.listdir(data_dir):
    print(image_class)
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        img_preprocessed = load_and_preprocess_image(image_path)
        if img_preprocessed is not None:
            X_data.append(img_preprocessed)
            y_data.append(image_class)

In [ ]:
label_encoder = LabelEncoder()
y_data_encoded = label_encoder.fit_transform(y_data)

# Shuffle and split the data into training, validation, and test sets
X_data, y_data_encoded = shuffle(X_data, y_data_encoded)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_encoded, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalize the image data using OneDAL
X_train_normalized = np.array([normalization_zscore(X_train[i].astype(np.float64)).compute().values for i in range(len(X_train))])
X_val_normalized = np.array([normalization_zscore(X_val[i].astype(np.float64)).compute().values for i in range(len(X_val))])
X_test_normalized = np.array([normalization_zscore(X_test[i].astype(np.float64)).compute().values for i in range(len(X_test))])

Model Making


In [ ]:
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_normalized, y_train, epochs=20, validation_data=(X_val_normalized, y_val))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Save the trained model
model.save('esophagelvarice_intel_efficient.h5')

Decision Making

In [ ]:
new_model=load_model(os.path.join('Models','esophagealvarice.h5'))
probab=new_model.predict(np.expand_dims(resizeimg/255,0))
if probab > 0.7:
    print("High risk varice! Banding is definitely required.")
elif probab > 0.5:
    print("High risk varice with chances of bleeding. Final diagnosis by doctor required.")
else:
    print("Low risk varice with low chances of bleeding.")